In [130]:
import gzip
import json
import os
import tarfile
import pandas as pd

from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu

import load_utils as lu
import sniffbytes as snif
import scanzip as szip

tardir = '/media/francois/seagate_1tb/cimaq_dicoms_20190901'

In [131]:
def sort_cimaq_20190901(src_path):
    while True:
        
        tarfiles = lu.loadfiles(sorted(apath for apath in
                                       lu.loadimages(tardir)
                                       if apath.endswith('.tar')))
        others = lu.loadfiles(sorted(apath for apath in
                                     lu.loadimages(tardir)
                                     if not apath.endswith('.tar')))
        others['fobj'] = [pd.read_csv(fpath, sep = '\t')
                          for fpath in others.fpaths]
        tarfiles['filename'] = [row[1]['filename']+row[1]['ext']
                                for row in tarfiles.iterrows()]

        l00 = others.iloc[0].fobj.sort_values('Archive')
        l02 = others.iloc[2].fobj.sort_values('Archive')
        l02['present'] = [arcname in tarfiles.filename.values.tolist()
                          for arcname in l02.Archive]

        l03 = l02.iloc[[row[0] for row in l02.iterrows()
                        if row[1]['QC'] == 'Pass']]
        l03 = l03.drop([row[0] for row in l03.iterrows()
                         if row[1]['QC'] != 'Pass'])

        l04 = dict(((grp[0], (grp[1],
                              l03.groupby('Archive').get_group(grp[1])))
                    for grp in enumerate(l03.groupby('Archive').groups)))
        tarfiles = tarfiles.sort_values('filename')
        tarfiles = tarfiles.iloc[[row[0] for row in tarfiles.iterrows()
                                 if row[1]['filename'] in
                                  l03.Archive.values.tolist()]]

        tarfiles['contents'] =[pd.concat([row[1] for row in l03.iterrows()
                                if row[1]['Archive'] \
                                == trow[1]['filename']],
                                         axis = 1).to_dict()
                               for trow in tarfiles.iterrows()]
        jsontar = json.dumps(tarfiles.drop(['parent', 'fpaths', 'ext'],
                                           axis = 1).to_dict())
        with open(xpu('~/cimaq_dicoms_20190901_contents.json'),
                      'w') as jfile:
            jfile.write(jsontar)
        jfile.close()
        break


,filename,ext,parent,fpaths
0,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
1,DCM_2015-05-12_CIMAQ_5522727_267168_V3.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
2,DCM_2015-05-12_CIMAQ_6120847_611208_V3.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
3,DCM_2015-05-21_ImagingUpload-16-34-hOHT29.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
4,DCM_2015-05-22_ImagingUpload-17-1-E8S6zt.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
...,...,...,...,...
215,DCM_2019-05-30_ImagingUpload-11-7-uVrc3b.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
216,DCM_2019-06-14_ImagingUpload-8-59-_sXuXs.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
217,DCM_2019-07-04_ImagingUpload-10-55-rc0YGY.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...
218,DCM_2019-07-11_ImagingUpload-11-11-1Ytqb0.tar,.tar,cimaq_dicoms_20190901,/media/francois/seagate_1tb/cimaq_dicoms_20190...


In [145]:
pd.read_json(xpu('~/cimaq_dicoms_20190901_contents.json'))

,filename,contents
0,DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar,"{'0': {'PSCID': 8192087, 'CandID': 619278, 'Vi..."
1,DCM_2015-05-12_CIMAQ_5522727_267168_V3.tar,"{'31': {'PSCID': 5522727, 'CandID': 267168, 'V..."
2,DCM_2015-05-12_CIMAQ_6120847_611208_V3.tar,"{'10': {'PSCID': 6120847, 'CandID': 611208, 'V..."
4,DCM_2015-05-22_ImagingUpload-17-1-E8S6zt.tar,"{'1979': {'PSCID': 8212673, 'CandID': 914042, ..."
5,DCM_2015-05-22_ImagingUpload-17-5-szWxaX.tar,"{'1998': {'PSCID': 8827943, 'CandID': 845675, ..."
...,...,...
215,DCM_2019-05-30_ImagingUpload-11-7-uVrc3b.tar,"{'3237': {'PSCID': 4117134, 'CandID': 350555, ..."
216,DCM_2019-06-14_ImagingUpload-8-59-_sXuXs.tar,"{'3354': {'PSCID': 3967609, 'CandID': 932933, ..."
217,DCM_2019-07-04_ImagingUpload-10-55-rc0YGY.tar,"{'3280': {'PSCID': 4084560, 'CandID': 549026, ..."
218,DCM_2019-07-11_ImagingUpload-11-11-1Ytqb0.tar,"{'3316': {'PSCID': 3874544, 'CandID': 983291, ..."


In [6]:
mytar = tarfile.open(tarpath, 'r')
# help(mytar)
mytar.__dict__

{'mode': 'r',
 '_mode': 'rb',
 '_extfileobj': False,
 'name': '/media/francois/seagate_1tb/cimaq_dicoms_20190901/DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar',
 'fileobj': <_io.BufferedReader name='/media/francois/seagate_1tb/cimaq_dicoms_20190901/DCM_2015-05-08_ImagingUpload-14-14-grO2zP.tar'>,
 'errors': 'surrogateescape',
 'pax_headers': {},
 'copybufsize': None,
 'closed': False,
 'members': [<TarInfo 'ImagingUpload-14-14-grO2zP.meta' at 0x7fa90633cdc0>],
 '_loaded': False,
 'offset': 174080,
 'inodes': {},
 'firstmember': <TarInfo 'ImagingUpload-14-14-grO2zP.meta' at 0x7fa90633cdc0>}